# 静态量化

参考：[静态量化](https://pytorch.org/tutorials/advanced/static_quantization_tutorial.html)

本教程展示了如何进行训练后的静态量化，并说明了两个更高级的技术——逐通道量化和感知量化的训练——以进一步提高模型的准确性。注意，量化目前 CPU 支持更好，所以在本教程中我们不会使用 GPU/CUDA。在本教程结束时，您将看到 PyTorch 中的量化如何在提高速度的同时显著降低模型大小。此外，您还将看到如何轻松地应用这里所展示的一些高级量化技术，从而使您的量化模型比其他方法获得更少的精度。

In [1]:
import torch
import torch.nn as nn

import time

# 设置 warnings
import warnings
warnings.filterwarnings(
    action='ignore',
    category=DeprecationWarning,
    module=r'.*'
)
warnings.filterwarnings(
    action='default',
    module=r'torch.quantization'
)

# 为可重复的结果指定随机种子
torch.manual_seed(27)

from mod import load_mod
load_mod()

通过几个显著的修改来启用量化：

- 用 {class}`torch.nn.quantized.FloatFunctional` 替换加法
- 在网络的开头和结尾分别插入 {class}`torch.ao.quantization.QuantStub` 和 {class}`torch.ao.quantization.DeQuantStub`

## 辅助函数

接下来，我们定义几个[帮助函数](https://github.com/pytorch/examples/blob/master/imagenet/main.py)来帮助评估模型。

In [2]:
from helper import accuracy, evaluate, print_size_of_model, AverageMeter

## 定义数据集和数据加载器

作为最后一个主要的设置步骤，我们为训练和测试集定义了数据加载器。

In [3]:
from xinet import CV

batch_size = 128
train_iter, test_iter = CV.load_data_cifar10(batch_size=batch_size)

Files already downloaded and verified
Files already downloaded and verified


## 模型载入和配置

In [4]:
from torch import nn

from torchvision.models.quantization import resnet18


class ResnetCustom:
    def __init__(self, num_class):
        self.num_class = num_class

    def combined_model(self, pretrained=True,
                       progress=True,
                       quantize=True):
        # 我们需要 `fc` 中过滤器的数量，以便将来使用。
        # 这里每个输出样本的大小设置为 2。
        # 或者，它可以推广到 nn.Linear(num_ftrs, len(class_names))。
        model_fe = resnet18(pretrained=pretrained,
                            progress=progress,
                            quantize=quantize)
        num_ftrs = model_fe.fc.in_features
        # 步骤1：分离特征提取器
        model_fe_features = nn.Sequential(
            model_fe.quant,  # 量化 input
            model_fe.conv1,
            model_fe.bn1,
            model_fe.relu,
            model_fe.maxpool,
            model_fe.layer1,
            model_fe.layer2,
            model_fe.layer3,
            model_fe.layer4,
            model_fe.avgpool,
            model_fe.dequant,  # 反量化 output
        )

        # 步骤2：创建一个新的“头”
        new_head = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(num_ftrs, self.num_class),
        )
        # 步骤3：合并，不要忘记量化 stubs
        new_model = nn.Sequential(
            model_fe_features,
            nn.Flatten(1),
            new_head,
        )
        return new_model